In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [2]:
df = pd.read_csv('../dataset/dataset.csv', na_values=["Unknown", "Best Price"])

In [3]:
saq = pd.read_csv('../dataset/price.csv')

In [4]:
df.shape

(9280, 12)

In [5]:
saq.shape

(195, 8)

In [6]:
df.head()

,URL,Name:,Company:,Type:,Distillation:,ABV:,Location,Score,nRatings,Price,Aged:,Distilled:
0,https://rumratings.com/rum/316-diplomatico-res...,Reserva Exclusiva,Diplomatico,Aged,Pot Still,40.0%,Venezuela,7.9,2802,44.99,NaN,NaN
1,https://rumratings.com/rum/853-ron-zacapa-23-s...,23 Solera,Ron Zacapa,Aged,Column Still (1-4),40.0%,Guatemala,7.5,2188,53.99,NaN,NaN
2,https://rumratings.com/rum/482-kraken-black-sp...,Black Spiced,Kraken,Spiced,Column Still (1-4),40.0%,Trinidad and Tobago,6.0,1731,29.99,NaN,NaN
3,https://rumratings.com/rum/654-plantation-xo-2...,XO 20th Anniversary,Plantation,Aged,Pot and Column Still,40.0%,Multiple,8.4,1697,49.99,NaN,NaN
4,https://rumratings.com/rum/367-el-dorado-12-year,12-Year,El Dorado,Aged,Pot and Column Still,40.0%,Guyana,7.6,1341,40.99,12.0,NaN


In [7]:
# rename column names and remove URL column
df = df.iloc[:, 1:]
df = df.rename(columns={'Name:': 'Name', 
                        'Company:': 'Company', 
                        'Type:': 'Type', 
                        'Distillation:': 'Distillation', 
                        'ABV:': 'ABV', 
                        'Aged:': 'Aged',
                        'Distilled:': 'Distilled'})

In [8]:
# drop the rums that have no ratings
df = df.drop(df[pd.isna(df.nRatings)].index)

In [9]:
df.shape

(7603, 11)

In [10]:
# drop duplicates
df = df.drop_duplicates()

In [11]:
df.shape

(5178, 11)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5178 entries, 0 to 7602
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          5177 non-null   object 
 1   Company       5178 non-null   object 
 2   Type          5176 non-null   object 
 3   Distillation  1145 non-null   object 
 4   ABV           3886 non-null   object 
 5   Location      5178 non-null   object 
 6   Score         5178 non-null   float64
 7   nRatings      5178 non-null   object 
 8   Price         139 non-null    float64
 9   Aged          2012 non-null   float64
 10  Distilled     1268 non-null   float64
dtypes: float64(4), object(7)
memory usage: 485.4+ KB


In [13]:
df['Type'].unique()

array(['Aged', 'Spiced', 'Gold', 'Light', 'Flavored', 'Dark', 'Agricole',
       'Cachaca', nan], dtype=object)

In [14]:
df[df['Type'].isnull()]

,Name,Company,Type,Distillation,ABV,Location,Score,nRatings,Price,Aged,Distilled
995,5-Year,Legionar,NaN,NaN,40.0%,Czechia,4.9,18,NaN,5.0,NaN
5160,Silver,Players,NaN,NaN,NaN,Netherlands,3.0,1 rating,NaN,NaN,NaN


In [15]:
# Check if these two products exist in saq
saq[((saq['Type'] == '5-Year') & (saq['Company'] == 'Legionar')) | (saq['Type'] == 'Silver') & (saq['Company'] == 'Players')]

,Request,Company,Name,Full Name,Type,Size,Country,Price


In [16]:
# remove these two products
df = df.drop(df[df['Type'].isnull()].index)

In [17]:
# Remove Cachaca and Agricole, focus on classic rum
df
df = df.drop(df[(df['Type'] == 'Cachaca') | (df['Type'] == 'Agricole')].index)

In [19]:
saq['Type'].unique()

array(['Brown rum', 'Spiced rum', 'Amber rum', 'White rum',
       'Rum and coconut liqueur', 'Flavoured rum (pineapple)',
       'Black rum', 'Flavoured rum (lemon)', 'Golden rum',
       'Flavoured rum', 'Flavoured rum (coconut)',
       'Flavoured rum (coffee)', 'Flavoured spiced rum (coffee)',
       'Flavoured rum (mango and pineapple)', 'Flavoured rum (raspberry)',
       'Not applicable rum', 'Flavoured spiced rum (chocolate)'],
      dtype=object)

In [20]:
saq[saq['Type'] == 'Rum and coconut liqueur']

,Request,Company,Name,Full Name,Type,Size,Country,Price
16,Malibu Coconut,Malibu,Coconut,Malibu,Rum and coconut liqueur,750 ml,Canada,26.85


In [21]:
saq[saq['Type'] == 'Not applicable rum']

,Request,Company,Name,Full Name,Type,Size,Country,Price
166,Bacardi Grapefruit,Bacardi,Grapefruit,Bacardi Grapefruit,Not applicable rum,750 ml,United States,25.10


In [22]:
saq.loc[(saq['Type'] == 'Rum and coconut liqueur') | (saq['Type'] == 'Not applicable rum'), 'Type'] = 'Flavoured rum'

In [23]:
df['Type'].unique()

array(['Aged', 'Spiced', 'Gold', 'Light', 'Flavored', 'Dark'],
      dtype=object)

In [24]:
saq['Type'].unique()

array(['Brown rum', 'Spiced rum', 'Amber rum', 'White rum',
       'Flavoured rum', 'Flavoured rum (pineapple)', 'Black rum',
       'Flavoured rum (lemon)', 'Golden rum', 'Flavoured rum (coconut)',
       'Flavoured rum (coffee)', 'Flavoured spiced rum (coffee)',
       'Flavoured rum (mango and pineapple)', 'Flavoured rum (raspberry)',
       'Flavoured spiced rum (chocolate)'], dtype=object)

In [26]:
def get_type(src):
    if 'Flavoured' in src:
        return 'Flavoured'
    elif 'Brown' in src:
        return 'Brown'
    elif 'Spiced' in src:
        return 'Spiced'
    elif 'Amber' in src:
        return 'Amber'
    elif 'White' in src:
        return 'White'
    elif 'Black' in src:
        return 'Black'
    elif 'Golden' in src:
        return 'Golden'
    else:
        return 'Not Specified'

saq['Type_simplified'] = saq['Type'].apply(get_type)

In [27]:
saq['Type_simplified'].unique()

array(['Brown', 'Spiced', 'Amber', 'White', 'Flavoured', 'Black',
       'Golden'], dtype=object)

In [28]:
df['Type'].unique()

array(['Aged', 'Spiced', 'Gold', 'Light', 'Flavored', 'Dark'],
      dtype=object)

In [29]:
df.rename(columns={'Type': 'Type_org'}, inplace=True)
saq.rename(columns={'Type': 'Type_org'}, inplace=True)

In [30]:
# Map type categories
type_dict = {'Aged': 'Brown', 'Spiced': 'Spiced', 'Gold': 'Golden', 'Light': 'White', 'Flavored': 'Flavoured', 'Dark': 'Black'}

def map_type(row):
    match_row = saq[(saq['Name'] == row['Name']) & (saq['Company'] == row['Company'])]
    
    if not match_row.empty:
        return match_row['Type_simplified'].values[0]
    else:
        return type_dict.get(row['Type_org'], None)
    
df['Type'] = df.apply(map_type, axis=1)

In [31]:
df['Type'].unique()

array(['Brown', 'Spiced', 'Amber', 'White', 'Flavoured', 'Black',
       'Golden'], dtype=object)

In [32]:
df.columns

Index(['Name', 'Company', 'Type_org', 'Distillation', 'ABV', 'Location',
       'Score', 'nRatings', 'Price', 'Aged', 'Distilled', 'Type'],
      dtype='object')

In [33]:
saq.columns

Index(['Request', 'Company', 'Name', 'Full Name', 'Type_org', 'Size',
       'Country', 'Price', 'Type_simplified'],
      dtype='object')

In [34]:
def get_unit_price(row):
    size_numeric = float(re.search(r'(\d+\.\d+|\d+)', row['Size']).group())

    size_units = re.search(r'([a-zA-Z]+)', row['Size']).group()

    conversion_factors = {'L': 1, 'ml': 1/1000}
    size_numeric = size_numeric * conversion_factors.get(size_units, 1)

    price_cleaned = float(row['Price'].replace(',', ''))

    unit_price = round(price_cleaned / size_numeric, 2)

    return unit_price


saq['Price_unit'] = saq.apply(get_unit_price, axis=1)

In [35]:
saq.head()

,Request,Company,Name,Full Name,Type_org,Size,Country,Price,Type_simplified,Price_unit
0,Diplomatico Reserva Exclusiva,Diplomatico,Reserva Exclusiva,Diplomatico Reserva Exclusiva,Brown rum,750 ml,Venezuela,63.50,Brown,84.67
1,Ron Zacapa 23 Solera,Ron Zacapa,23 Solera,Ron Zacapa Centenario Sistema Solera 23 Gran R...,Brown rum,750 ml,Guatemala,86.50,Brown,115.33
2,Kraken Black Spiced,Kraken,Black Spiced,The Kraken Black Spiced,Spiced rum,1.14 L,United States,48.25,Spiced,42.32
3,Plantation XO 20th Anniversary,Plantation,XO 20th Anniversary,Plantation 20Th Anniversary XO Barbados,Brown rum,700 ml,Barbados,72.50,Brown,103.57
4,El Dorado 12-Year,El Dorado,12-Year,El Dorado 12 ans Demerara,Brown rum,750 ml,Guyana,45.00,Brown,60.00


In [36]:
df.head()

,Name,Company,Type_org,Distillation,ABV,Location,Score,nRatings,Price,Aged,Distilled,Type
0,Reserva Exclusiva,Diplomatico,Aged,Pot Still,40.0%,Venezuela,7.9,2802,44.99,NaN,NaN,Brown
1,23 Solera,Ron Zacapa,Aged,Column Still (1-4),40.0%,Guatemala,7.5,2188,53.99,NaN,NaN,Brown
2,Black Spiced,Kraken,Spiced,Column Still (1-4),40.0%,Trinidad and Tobago,6.0,1731,29.99,NaN,NaN,Spiced
3,XO 20th Anniversary,Plantation,Aged,Pot and Column Still,40.0%,Multiple,8.4,1697,49.99,NaN,NaN,Brown
4,12-Year,El Dorado,Aged,Pot and Column Still,40.0%,Guyana,7.6,1341,40.99,12.0,NaN,Brown


In [37]:
merged = pd.merge(df, saq, how='left', on=['Name', 'Company'], suffixes=('_df', '_saq'))

In [51]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4592 entries, 0 to 4591
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             4591 non-null   object 
 1   Company          4592 non-null   object 
 2   Type_org_df      4592 non-null   object 
 3   Distillation     693 non-null    object 
 4   ABV              3334 non-null   object 
 5   Location         4592 non-null   object 
 6   Score            4592 non-null   float64
 7   nRatings         4592 non-null   object 
 8   Price_df         137 non-null    float64
 9   Aged             1716 non-null   float64
 10  Distilled        1062 non-null   float64
 11  Type             4592 non-null   object 
 12  Request          194 non-null    object 
 13  Full Name        194 non-null    object 
 14  Type_org_saq     194 non-null    object 
 15  Size             194 non-null    object 
 16  Country          194 non-null    object 
 17  Price_saq     

In [52]:
merged = merged.drop(['Type_org_df', 'Type_simplified', 'Type_org_saq', 'Request', 'Price_df', 'Size', 'Price_saq'], axis=1)

In [53]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4592 entries, 0 to 4591
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          4591 non-null   object 
 1   Company       4592 non-null   object 
 2   Distillation  693 non-null    object 
 3   ABV           3334 non-null   object 
 4   Location      4592 non-null   object 
 5   Score         4592 non-null   float64
 6   nRatings      4592 non-null   object 
 7   Aged          1716 non-null   float64
 8   Distilled     1062 non-null   float64
 9   Type          4592 non-null   object 
 10  Full Name     194 non-null    object 
 11  Country       194 non-null    object 
 12  Price_unit    194 non-null    float64
dtypes: float64(4), object(9)
memory usage: 466.5+ KB


In [93]:
df.to_csv("../dataset/rum_ratings_cleaned.csv", index=False)

In [94]:
saq.to_csv("../dataset/saq_cleaned.csv", index=False)

In [54]:
merged.to_csv("../dataset/merged_cleaned.csv", index=False)